# **1° Passo:** Instalando as Bibliotecas Necessárias



In [ ]:
# Installing the numpy, netcdf4, boto3 and gdal libraries
!pip install -q cartopy boto3 gdal salem rasterio pyproj geopandas descartes

# Download dos arquivos auxiliares
!wget -c https://raw.githubusercontent.com/evmpython/imagens_GOES/main/input/utilities_goes19.py
!wget -c https://raw.githubusercontent.com/evmpython/imagens_GOES/main/input/ir.cpt

# Monta drive
from google.colab import drive
drive.mount('/content/drive')

# Caminho do diretório
dir = '/content/drive/MyDrive/2-PESQUISA/0_GLM/estudos_de_caso/2025-05-27e28-FRENTEFRIA'

# Diretório de Saída
import os
dir_output = f'{dir}/output'
os.makedirs(dir_output, exist_ok=True)

#**Script 01** - Projeção Satélite e Imagem Escala de Cinza

In [ ]:
#========================================================================================================================#
#                                          IMPORTAÇÃO DAS BIBLIOTECAS
#========================================================================================================================#
import xarray as xr
import matplotlib.pyplot as plt
import cartopy, cartopy.crs as ccrs
from datetime import datetime
from utilities_goes19 import download_CMI
import os

#========================================================================================================================#
#                                        CRIA DIRETÓRIO DE ENTRADA E SAÍDA
#========================================================================================================================#
input = "/content/input"; os.makedirs(input, exist_ok=True)
output = dir_output

#========================================================================================================================#
#                                               DOWNLOAD DO ARQUIVO
#========================================================================================================================#
# data de processamento
yyyymmddhhmn = '202505272340'

# canal do ABI
band = '13'

# download do arquivo (CMI: "Cloud and Moisture Imagery" Product)
file_name = download_CMI(yyyymmddhhmn, band, input)

# caminho do arquivo que foi baixado
path = f'{input}/{file_name}.nc'

#========================================================================================================================#
#                                               LEITURA DO ARQUIVO
#========================================================================================================================#
# abre a imagem
data = xr.open_dataset(path)

#========================================================================================================================#
#                                                 PLOTA A IMAGEM
#========================================================================================================================#
# tamanho da figura (largura x altura em polegadas)
plt.figure(figsize=(10,10))

# usa a projeção geoestacionária do cartopy
# para o GOES-16: longitude central: -75.0 / altura do satellite: 35786023.0
# a extensão do Full Disk: (metade dos pixels full disk) X (tamanho do pixel em radianos) X (altura do satélite em metros) => 2712 * 0.000056 * 35786023.0 = 5434894.67527
ax = plt.axes(projection=ccrs.Geostationary(central_longitude=-75.0, satellite_height=35786023.0))
img_extent = (-5434894.67527, 5434894.67527, -5434894.67527, 5434894.67527)

# linhas costeiras, bordas e linhas de grade do mapa
ax.coastlines(resolution='10m', color='white', linewidth=0.8)
ax.add_feature(cartopy.feature.BORDERS, edgecolor='white', linewidth=0.5)
ax.gridlines(color='white', alpha=0.5, linestyle='--', linewidth=0.5)

# escala de cores
colormap = "gray_r" # escala de cores na ordem reversa - branco para preto para o canal do IR

# plota imagem
img = ax.imshow(data['CMI'] - 273.15, origin='upper', vmin=-80, vmax=40, extent=img_extent, cmap=colormap)

# barra de cores
plt.colorbar(img, label='Temperatura de Brilho (°C)', extend='both', orientation='vertical', pad=0.05, fraction=0.05)

# leitura da data/horário do arquivo NetCDF como uma string
date = (datetime.strptime(data.time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ')).strftime('%Y-%m-%d %H:%M UTC')

# título da figura
plt.title(f'GOES-19 Banda 13 (10.35 µm)\n{date}', fontweight='bold', fontsize=10, loc='left')
plt.title('Full Disk', fontsize=10, loc='right')

# salva figura
plt.savefig(f'{output}/script_1_{date.replace(" ", "_")}.jpg')

# mostra figura na tela
plt.show()

#**Script 02** - Projeção Satélite e Imagem Realçada



In [ ]:
%%time
#========================================================================================================================#
#                                          IMPORTAÇÃO DAS BIBLIOTECAS
#========================================================================================================================#
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib import cm
import cartopy, cartopy.crs as ccrs
from datetime import datetime
from utilities_goes19 import download_CMI, remap, loadCPT
import os

#========================================================================================================================#
#                                        CRIA DIRETÓRIO DE ENTRADA E SAÍDA
#========================================================================================================================#
input = "/content/input"; os.makedirs(input, exist_ok=True)
output = dir_output

#========================================================================================================================#
#                                               DOWNLOAD DO ARQUIVO
#========================================================================================================================#
# data de processamento
yyyymmddhhmn = '202505272340'

# canal do ABI
band = '13'

# download do arquivo (CMI: "Cloud and Moisture Imagery" Product)
file_name = download_CMI(yyyymmddhhmn, band, input)

# caminho do arquivo que foi baixado
path = f'{input}/{file_name}.nc'

#========================================================================================================================#
#                                               LEITURA DO ARQUIVO
#========================================================================================================================#
# abre a imagem
data = xr.open_dataset(path)

#========================================================================================================================#
#                                                 PLOTA A IMAGEM
#========================================================================================================================#
# tamanho da figura (largura x altura em polegadas)
plt.figure(figsize=(10,10))

# usa a projeção geoestacionária do cartopy
# para o GOES-16: longitude central: -75.0 / altura do satellite: 35786023.0
# a extensão do Full Disk: (metade dos pixels full disk) X (tamanho do pixel em radianos) X (altura do satélite em metros) => 2712 * 0.000056 * 35786023.0 = 5434894.67527
ax = plt.axes(projection=ccrs.Geostationary(central_longitude=-75.0, satellite_height=35786023.0))
img_extent = (-5434894.67527, 5434894.67527, -5434894.67527, 5434894.67527)

# linhas costeiras, bordas e linhas de grade do mapa
ax.coastlines(resolution='10m', color='white', linewidth=0.8)
ax.add_feature(cartopy.feature.BORDERS, edgecolor='white', linewidth=0.5)
ax.gridlines(color='white', alpha=0.5, linestyle='--', linewidth=0.5)

# converte o arquivo CPT para ser usado em Python
cpt = loadCPT('ir.cpt')
colormap = cm.colors.LinearSegmentedColormap('cpt', cpt)

# plota imagem
img = ax.imshow(data['CMI'] - 273.15, origin='upper', vmin=-103.0, vmax=84, extent=img_extent, cmap=colormap)

# barra de cores
plt.colorbar(img, label='Temperatura de Brilho (°C)', extend='both', orientation='vertical', pad=0.05, fraction=0.05)

# leitura da data/horário do arquivo NetCDF como uma string
date = (datetime.strptime(data.time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ')).strftime('%Y-%m-%d %H:%M UTC')

# título da figura
plt.title(f'GOES-19 Banda 13 (10.35 µm)\n{date}', fontweight='bold', fontsize=10, loc='left')
plt.title('Full Disk', fontsize=10, loc='right')

# salva figura
plt.savefig(f'{output}/script_2_{date.replace(" ", "_")}.jpg')

# mostra figura na tela
plt.show()

#**Script 03** - Projeção Satélite (Imagem Realçada) + Total de Flashes do GLM



In [ ]:
#========================================================================================================================#
#                                               REQUIRED MODULES
#========================================================================================================================#
import xarray as xr                                                         # Work with multidimensional arrays
import matplotlib.pyplot as plt                                             # Plotting library
from matplotlib import cm                                                   # Colormap handling utilities
import cartopy, cartopy.crs as ccrs                                         # Plot maps
import cartopy.io.shapereader as shpreader                                  # Read shapefiles
from datetime import timedelta, datetime                                    # Basic Dates and time types
from utilities_goes19 import download_CMI, download_GLM, remap, loadCPT     # Our own utilities
import numpy as np                                                          # Scientific computing with Python
import os                                                                   # Miscellaneous operating system interfaces
import pandas as pd                                                         # Work with dataframes

#========================================================================================================================#
#                                        INPUT AND OUTPUT DIRECTORIES
#========================================================================================================================#
input = "/content/input"; os.makedirs(input, exist_ok=True)
output = dir_output

#========================================================================================================================#
#                                             DOWNLOAD THE ABI FILE
#========================================================================================================================#
# Datetime to process
yyyymmddhhmn = '202505272340'

# Channel
band = '13'

# Download the file
file_name = download_CMI(yyyymmddhhmn, band, input)

# Path of the downloaded file
path = f'{input}/{file_name}.nc'

#========================================================================================================================#
#                                         REPROJECT AND READ THE ABI DATA
#========================================================================================================================#
# Desired extent (min lon, min lat, max lon, max lat)
extent = [-65, -35, -45, -15]

# Call the reprojection funcion (file, variable, extent, resolution)
grid = remap(path, 'CMI', extent, 2)

# Read the data returned by the function and convert to °C
data = grid.ReadAsArray() - 273.15

#========================================================================================================================#
#                                                    GET GLM DATA
#========================================================================================================================#
# Initialize arrays for latitude and longitude of flashes
lats_flash = np.array([])
lons_flash = np.array([])

# Time and date references
yyyy = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%Y')
mm = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%m')
dd = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%d')
hh = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%H')
mn = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%M')
date_ini = str(datetime(int(yyyy),int(mm),int(dd),int(hh),int(mn)))
date_end = str(datetime(int(yyyy),int(mm),int(dd),int(hh),int(mn)) + timedelta(minutes=10))
date_loop = date_ini

# GLM accumulation loop
while (date_loop <= date_end):

    # Date structure
    yyyymmddhhmnss = datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S').strftime('%Y%m%d%H%M%S')

    # Download the file
    file_glm = download_GLM(yyyymmddhhmnss, input)

    # Read the file
    glm_20s = xr.open_dataset(f'{input}/{file_glm}.nc')

    # Append lats / longs
    lats_flash = np.append(lats_flash, glm_20s['flash_lat'][:])
    lons_flash = np.append(lons_flash, glm_20s['flash_lon'][:])

    # Close the file
    glm_20s.close()

    # Increment the date_loop
    date_loop = str(datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S') + timedelta(seconds=20))

# Select the flashes inside the region
# put the flashes into dataframe
data_flash = {'lat': lats_flash, 'lon': lons_flash}
df = pd.DataFrame(data_flash)

# select the flashes from region of interest
df_flash_filtered = df[ (df['lon'] > extent[0]) & (df['lon'] < extent[2]) & (df['lat'] > extent[1]) & (df['lat'] < extent[3])]

# transform from dataframe to array
lons_flash_filtered, lats_flash_filtered = df_flash_filtered['lon'].values, df_flash_filtered['lat'].values

#========================================================================================================================#
#                                                 PLOT THE IMAGE
#========================================================================================================================#
# Choose the plot size (width x height, in inches)
plt.figure(figsize=(14,11))

# Use the Geostationary projection in cartopy
ax = plt.axes(projection=ccrs.PlateCarree())

# Define the data extent
img_extent = [extent[0], extent[2], extent[1], extent[3]]

# Converts a CPT file to be used in Python
cpt = loadCPT('ir.cpt')
colormap = cm.colors.LinearSegmentedColormap('cpt', cpt)

# Plot the image
vmin = -103.0; vmax = 84
img = ax.imshow(data, origin='upper', vmin=vmin, vmax=vmax, extent=img_extent, cmap=colormap, alpha=0.7, zorder=1)

# Plot the GLM Data
glm = plt.scatter(lons_flash_filtered, lats_flash_filtered, transform=ccrs.PlateCarree(), marker='o', s=15, facecolor='white', edgecolor='black',
                  linewidth=1, alpha=0.8, zorder=2, label=f'Flashes={str(len(lats_flash_filtered)).zfill(4)}')

# Add legend
ax.legend(loc='lower right', ncols=1, facecolor='white', frameon=True)

# Add coastlines, borders and gridlines
ax.coastlines(resolution='10m', color='white', linewidth=0.8)
ax.add_feature(cartopy.feature.BORDERS, edgecolor='white', linewidth=0.5)
gl = ax.gridlines(crs=ccrs.PlateCarree(), color='white', alpha=1.0, linestyle='--', linewidth=0.25, xlocs=np.arange(-180, 180, 4), ylocs=np.arange(-90, 90, 4), draw_labels=True)
gl.top_labels = False
gl.right_labels = False

# Plot states
shapefile = list(shpreader.Reader('https://github.com/evmpython/minicurso_nowcasting_CPAM2024/raw/main/shapefiles/BR_UF_2019.shp').geometries())
ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='white', facecolor='none', linewidth=1.0)

# Add a colorbar
plt.colorbar(img, label='Brightness Temperature (°C)', extend='both', orientation='horizontal', pad=0.05, fraction=0.05)

# Read the time/date from the NetCDF file metadata as a string
date = (datetime.strptime(xr.open_dataset(path).time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ')).strftime('%Y-%m-%d %H:%M UTC')

# Add a title
#plt.title(f'GOES-19 Band 13 (10.3 µm) + GLM Flashes\nABI: {date}', fontweight='bold', fontsize=10, loc='left')
#plt.title(f'GLM: {str(date_ini)} - {str(date_end)}', fontsize="10", loc="right")
plt.title(f'GOES-19 Band 13 (10.3 µm) + GLM Flashes', fontweight='bold', fontsize=12, loc='left', color='red')
plt.title(f'{date}', fontsize="10", loc="right", color='black')

# Save the image
plt.savefig(f'{output}/script_3_{date.replace(" ", "_")}.jpg', bbox_inches='tight', dpi=300)

# Show the image
plt.show()

#**Script 04** - Evolução Temporal da Temperatura de Brilho do IR e Flashes GLM



## **Mapa**

In [ ]:
#========================================================================================================================#
#                                               REQUIRED MODULES
#========================================================================================================================#
import xarray as xr                                              # Work with multidimensional arrays
import matplotlib.pyplot as plt                                  # Plotting library
from matplotlib import cm                                        # Colormap handling utilities
import cartopy, cartopy.crs as ccrs                              # Plot maps
import cartopy.io.shapereader as shpreader                       # Read shapefiles
from datetime import timedelta, datetime                         # Basic Dates and time types
from utilities_goes19 import download_CMI, download_GLM, remap, loadCPT # Our own utilities
import numpy as np                                               # Scientific computing with Python
import os                                                        # Miscellaneous operating system interfaces
import pandas as pd                                              # Work with dataframes
import geopandas as gpd                                          # Work with geodataframes
import salem                                                     # Work with shapefiles

#========================================================================================================================#
#                                             INITIAL DEFINITIONS
#========================================================================================================================#
# Create the input and output directories
input = "/content/input"; os.makedirs(input, exist_ok=True)
output = dir_output

# Channel
band = '13'

# Desired extent
lonmin, lonmax, latmin, latmax = -65, -45, -35, -15
extent = [lonmin, latmin, lonmax, latmax] # min lon, max lon, min lat, max lat

#========================================================================================================================#
#                                              LOOP OF IMAGES
#========================================================================================================================#
# Loop
for date_image in pd.date_range('202505272030', '202505281200', freq='10min'):

    #--------------------------------------------------------------------------#
    #                          TIME AND DATE
    #--------------------------------------------------------------------------#
    # Datetime to process
    yyyymmddhhmn = date_image.strftime('%Y%m%d%H%M') # '202404301300'

    # Time and data references
    yyyy = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%Y')
    mm = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%m')
    dd = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%d')
    hh = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%H')
    mn = datetime.strptime(yyyymmddhhmn, '%Y%m%d%H%M').strftime('%M')

    print('#=====================================================================================================#')
    print(f'                           PROCESSING THE IMAGE = {yyyy}-{mm}-{dd} {hh}{mn} UTC'                       )
    print('#=====================================================================================================#')

    # Download the file
    file_name = download_CMI(yyyymmddhhmn, band, input)

    # Path of the downloaded file
    path = f'{input}/{file_name}.nc'

    #--------------------------------------------------------------------------#
    #                    REPROJECT AND READ THE ABI DATA
    #--------------------------------------------------------------------------#

    # Call the reprojection funcion (file, variable, extent, resolution)
    grid = remap(path, 'CMI', extent, 2)

    # Read the data returned by the function and convert to °C
    data = grid.ReadAsArray() - 273.15

    #--------------------------------------------------------------------------#
    #                           GET THE GLM DATA
    #--------------------------------------------------------------------------#
    # Date of the current image and next image
    date_ini = str(datetime(int(yyyy),int(mm),int(dd),int(hh),int(mn)))
    date_end = str(datetime(int(yyyy),int(mm),int(dd),int(hh),int(mn)) + timedelta(minutes=10))
    date_loop = date_ini

    # GLM accumulation loop
    lats_flash, lons_flash = np.array([]), np.array([])
    while (date_loop <= date_end):

        # Date structure
        yyyymmddhhmnss = datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S').strftime('%Y%m%d%H%M%S')

        # Download the file
        file_glm = download_GLM(yyyymmddhhmnss, input)

        # Read the file
        glm_20s = xr.open_dataset(f'{input}/{file_glm}.nc')

        # Append lats / longs
        lats_flash = np.append(lats_flash, glm_20s['flash_lat'][:])
        lons_flash = np.append(lons_flash, glm_20s['flash_lon'][:])

        # Close the file
        glm_20s.close()

        # Increment the date_loop
        date_loop = str(datetime.strptime(date_loop, '%Y-%m-%d %H:%M:%S') + timedelta(seconds=20))
    #--------------------------------------------------------------------------#

    # Select the flashes inside the region
    # Put the flashes into dataframe
    data_flash = {'lat': lats_flash, 'lon': lons_flash}
    df = pd.DataFrame(data_flash)

    # Select the flashes from region of interest
    df_flash_filtered = df[ (df['lon'] > lonmin) & (df['lon'] < lonmax) & (df['lat'] > latmin) & (df['lat'] < latmax)]

    # Transform from dataframe to array
    lons_flash_filtered, lats_flash_filtered = df_flash_filtered['lon'].values, df_flash_filtered['lat'].values

    #--------------------------------------------------------------------------#
    #                           PLOT THE IMAGE
    #--------------------------------------------------------------------------#
    # Choose the plot size (width x height, in inches)
    plt.figure(figsize=(14,11))

    # Use the Geostationary projection in cartopy
    ax = plt.axes(projection=ccrs.PlateCarree())

    # Converts a CPT file to be used in Python
    cpt = loadCPT('ir.cpt')
    cmap = cm.colors.LinearSegmentedColormap('cpt', cpt)

    # Plot the image
    img = ax.imshow(data, origin='upper', vmin=-103.0, vmax=84, extent=[lonmin, lonmax, latmin, latmax], cmap=cmap, alpha=0.7, zorder=1)

    # Plot the GLM Data
    glm = plt.scatter(lons_flash_filtered, lats_flash_filtered, transform=ccrs.PlateCarree(), marker='o', s=15, facecolor='white', edgecolor='black', linewidth=1, alpha=0.8, zorder=2, label=f'Flashes={str(len(lats_flash_filtered)).zfill(4)}')

    # Add legend
    ax.legend(loc='lower right', ncols=1, facecolor='white', frameon=True)

    # Add gridlines
    gl = ax.gridlines(crs=ccrs.PlateCarree(), color='gray', alpha=1.0, linestyle='--', linewidth=0.25, xlocs=np.arange(-180, 180, 5), ylocs=np.arange(-90, 90, 5), draw_labels=True)
    gl.top_labels = False
    gl.right_labels = False

    # Plot States shapefile
    shapefile = list(shpreader.Reader('https://github.com/evmpython/minicurso_nowcasting_CPAM2024/raw/main/shapefiles/BR_UF_2019.shp').geometries())
    ax.add_geometries(shapefile, ccrs.PlateCarree(), edgecolor='white', facecolor='none', linewidth=1.0)

    # Add a colorbar
    plt.colorbar(img, label='Brightness Temperature (°C)', extend='both', orientation='horizontal', pad=0.05, fraction=0.05)

    # Read the time/date from the NetCDF file metadata as a string
    date = (datetime.strptime(xr.open_dataset(path).time_coverage_start, '%Y-%m-%dT%H:%M:%S.%fZ')).strftime('%Y-%m-%d %H:%M UTC')

    # Add a title
    plt.title(f'GOES-19 Band 13 (10.3 µm) + GLM Flashes', fontweight='bold', fontsize=12, loc='left', color='red')
    plt.title(f'{date}', fontsize="10", loc="right", color='black')

    # Save the image
    plt.savefig(f'{output}/script_4_{yyyy}-{mm}-{dd}_{hh}:{mn}_UTC.jpg', bbox_inches='tight', dpi=300)

    # Show the image (uncomment this to show each image)
    #plt.show()

## **Animação das imagens**

In [ ]:
# Import library
import imageio # Make animations
import glob    # To list files

# List the images for animation
files = sorted(glob.glob(f'{dir_output}/script_4*jpg'))

# Make the animation
images = []
for file in files:
    images.append(imageio.imread(file))

# Salve the animation
imageio.mimsave(f'{dir_output}/animacao.gif',
                images,
                duration=200,
                loop=0)

# Show the animation
print("\nAbrindo o GIF..\n")
from IPython.display import Image
Image(open(f'{dir_output}/animacao.gif','rb').read(), width=600)